# Process tomography

Because Pyquil offers the 'wavefunction' method, it is much easier to perform a quantum process tomography. It's necessary only to call 'wavefunction' $2^N$ times (where $N$ is the number of qubits).

In [1]:
import pylab as py
from pyquil.quil import Program
import pyquil.api as api
import pyquil.gates as gt
qvm = api.QVMConnection()
import itertools
try:
    import qutip as qp
except ModuleNotFoundError:
    qp = None
    

In [2]:
def do_process_tomography(program):
    qubits = sorted(program.get_qubits())
    dimension = 2**len(qubits)
    unitary = 1j*py.zeros((dimension,dimension))
    for column,initial_state in enumerate(itertools.product([False,True],repeat=len(qubits))):
        preparation = Program()
        for initial,qubit in zip(initial_state[::-1],qubits):
            if initial:
                preparation.inst(gt.X(qubit))
        wv_final = qvm.wavefunction(preparation + program)
        unitary[:,column] = py.array(wv_final.amplitudes)
    
    return unitary

# Automated testing


In [3]:
import unittest

class TestProcessTomography(unittest.TestCase):
    """Example of how to use unittest in Jupyter."""
    
    def test_hadamard(self):
        circuit = Program([gt.H(0)])
        unitary = qp.Qobj([[1,1],[1,-1]]) / py.sqrt(2)
        tomography = do_process_tomography(circuit)
        error = unitary.dag() * qp.Qobj(tomography,dims=unitary.dims)
        eigenvalues = error.eigenenergies()
        eigenvalues /= eigenvalues[0]
        assert all([abs(x-1)<1e-12 for x in eigenvalues])
        
    def test_CNOT(self):
        circuit = Program([gt.H(1),gt.CZ(0,1),gt.H(1)])
        unitary = qp.Qobj([[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]]) / py.sqrt(2)
        tomography = do_process_tomography(circuit)
        error = unitary.dag() * qp.Qobj(tomography,dims=unitary.dims)
        eigenvalues = error.eigenenergies()
        eigenvalues /= eigenvalues[0]
        assert all([abs(x-1)<1e-12 for x in eigenvalues])
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 1.255s

OK
